![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)
# Pipeline Notebook - AutoAI Notebook v1.18.4

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimal results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.10 and scikit-learn 1.1.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data with train and test data split](#read)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
&nbsp;&nbsp;[Forecast](#fore)<br>
**[Store the model](#saving)**<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-ts-libs,
 - scikit-learn


In [1]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U 'autoai-ts-libs>=1.2.0,<4.0' | tail -n 1
!pip install -U scikit-learn==1.1.1 | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [2]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='7c3ecd4f-2708-4870-9de5-788be6c5496d'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/f0f38b4e-c962-4e54-94fc-dcf364bb208d/wml_data/327ca112-2310-46b3-98f1-201885ecfe84/data/autoai-ts',
        model_location='auto_ml/f0f38b4e-c962-4e54-94fc-dcf364bb208d/wml_data/327ca112-2310-46b3-98f1-201885ecfe84/data/autoai-ts/model.zip',
        training_status='auto_ml/f0f38b4e-c962-4e54-94fc-dcf364bb208d/wml_data/327ca112-2310-46b3-98f1-201885ecfe84/training-status.json'
    )
)

The following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [3]:
experiment_metadata = dict(
    prediction_type='timeseries',
    prediction_columns=['precipitation', 'temp_max', 'temp_min', 'wind'],
    csv_separator=',',
    holdout_size=20,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    timestamp_column_name='date',
    backtest_num=4,
    pipeline_type='all',
    customized_pipelines=[],
    lookback_window=16,
    forecast_window=5,
    max_num_daub_ensembles=3,
    feature_columns=['precipitation', 'temp_max', 'temp_min', 'wind'],
    future_exogenous_available=True,
    gap_len=0,
    deployment_url='https://us-south.ml.cloud.ibm.com',
    project_id='2cb847c0-a98a-42d4-93f9-4d49c25d8909',
    numerical_imputation_strategy=['FlattenIterative', 'Linear', 'Cubic', 'Previous']
)

## Set `n_jobs` parameter to the number of available CPUs

In [4]:
import os, ast
CPU_NUMBER = -1
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: Provide the IBM Cloud apikey, For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [5]:
api_key = 'PUT_YOUR_APIKEY_HERE'

In [6]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

In [8]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(wml_client)



Error getting IAM Token.
Reason: <Response [400]>


WMLClientError: Error getting IAM Token.
Reason: <Response [400]>

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data with train and test data split

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`

In [9]:
train_X, test_X, train_y, test_y = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=False)

Cannot read saved remote data before fit.


CannotReadSavedRemoteDataBeforeFit: Cannot read saved remote data before fit.

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [ ]:
from autoai_ts_libs.utils.ts_pipeline import TSPipeline
from autoai_ts_libs.transforms.imputers import linear
from autoai_ts_libs.sklearn.autoai_ts_pipeline import AutoaiTSPipeline
from autoai_ts_libs.sklearn.mvp_windowed_transformed_target_estimators import (
    AutoaiWindowTransformedTargetRegressor,
)
from autoai_ts_libs.sklearn.small_data_window_transformers import (
    SmallDataWindowTransformer,
)
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

#### Pipeline definition.

In [ ]:
linear = linear(missing_val_identifier=float("nan"))
small_data_window_transformer = SmallDataWindowTransformer(lookback_window=16)
simple_imputer = SimpleImputer(missing_values=float("nan"))
random_forest_regressor = RandomForestRegressor(
    n_estimators=500, max_depth=30, n_jobs=CPU_NUMBER, random_state=33
)
autoai_ts_pipeline_0 = AutoaiTSPipeline(
    steps=[
        ("WTX", small_data_window_transformer),
        ("imputer", simple_imputer),
        ("est", random_forest_regressor),
    ]
)
autoai_window_transformed_target_regressor = (
    AutoaiWindowTransformedTargetRegressor(
        feature_columns=[0, 1, 2, 3],
        lookback_window=16,
        prediction_horizon=5,
        random_state=33,
        regressor=autoai_ts_pipeline_0,
        short_name="WindowRandomForest",
        target_columns=[0, 1, 2, 3],
    )
)
autoai_ts_pipeline = AutoaiTSPipeline(
    steps=[("windowedttr", autoai_window_transformed_target_regressor)]
)
pipeline = TSPipeline(
    steps=[
        ("linear_imputer", linear),
        (
            "<class 'autoai_ts_libs.sklearn.autoai_ts_pipeline.AutoaiTSPipeline'>",
            autoai_ts_pipeline,
        ),
    ],
    feature_columns=[0, 1, 2, 3],
    prediction_horizon=5,
    target_columns=[0, 1, 2, 3],
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from autoai_ts_libs.utils.metrics import get_scorer

scorer = get_scorer("neg_avg_symmetric_mean_absolute_percentage_error")

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [10]:
pipeline.fit(train_X.values, train_y.values);

NameError: name 'pipeline' is not defined

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

In [ ]:
pipeline.predict(test_X.values)

<a id="forecast"></a>
## Forecast

Forecast the future values of targets.

In [ ]:
pipeline.predict()

### Forecasting with the new observations
Provide new observations values
For example:
 - `a,b` are both targets and features, taken from `experiment_metadata["prediction_columns"]`

```
import numpy as np

a_newObs = [12, 22, 35, 46]
b_newObs = [34, 23, 45, 34]
newObs = np.mat([a_newObs, b_newObs]).T
pipeline.predict(newObs)
```


<a id="saving"></a>
## Store the model

In this section you will learn how to store the retrained model.

In [ ]:
model_metadata = {
    wml_client.repository.ModelMetaNames.NAME: 'P1 - Pretrained AutoAI pipeline'
}

stored_model_details = wml_client.repository.store_model(model=pipeline, meta_props=model_metadata, experiment_metadata=experiment_metadata)

Inspect the stored model details.

In [ ]:
stored_model_details

<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/cpd/overview/relnotes-4.0.html) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___